# Project on hold
### Things to do:
(a): Update ant_colony_modified.py to output node names in the paths instead of coordinates and other modifications.

(b): Calculate all "reasonable" (may be all) paths to subnodes and compare CP costs. Reason being that this is not exactly a mTSP (multiple travelling salesmen problem) because there are limited connections between nodes (which leads to a sparse connection matrix) and I do not need a cycle containing all nodes. Also, this is an asymmetrical problem (going into the subnodes is where the profits are, not going from subnodes to main nodes).

(c): Modify algorithm to take CP costs into account so there are constraints.

(d): Worker constraints (stamina, work speed, skills, etc.)

(e): Once I get the algorithm working then I can expand the project past its limitations by web-scraping one of the online BDO databases (bdodae).

In [1]:
# Import 'os'
import os

# Get current working directory ('cwd')
cwd = os.getcwd()

# Import pandas, numpy, re (regular expressions)
import pandas as pd
import numpy as np
import re

In [5]:
# Import Black Desert Nodes and Resources
# Contains connection nodes as well as resource nodes
nodes = pd.read_excel(io = 'Black Desert Nodes and Resources.xlsx', sheet_name = 'List By region')

# Import final_node_names (clean)
# Contains the connection matrix between nodes (excluding islands)
# Created by manually inputting ones to indicate connection
final_node = pd.read_excel(io = 'final_node_names (clean).xlsx', sheet_name = 'Sheet1')

# Match column and row names (duplicate columns became *****.1 on import)
final_node['Name'] = final_node.columns[1:]

# Import Public Node Doc - Balzor (clean)
# Contains resource/subnodes
# Clean version updated using data from BDOdae: average yields of traces
# Clean Public Node Doc - Balzor : Northern Wheat Pl to Northern Wheat Plantation
balzor = pd.read_excel(io = 'Public Node Doc - Balzor (clean).xlsx', sheet_name = "Balzor's Node Chart")

In [6]:
# BDO map http://www.somethinglovely.net/bdo/

In [7]:
# Olivia == Olvia, Terrmian == Thermian, Mediah == Median, Port Epheria == Epheria, Catfishman Camp == Catfish Camp
# Bradie Forest == Bradie Fortress, Abandonned Quarry == Abandoned Quarry, Trina beacon Mounds == Trina Beacon Towers
# North/South Kaia Pier == North/South Kaia Ferry, Trina beacon Moudns == Trina Beacon Towers
# Bradie Forest == Bradie Fortress
# Norhern Media Plateau == Mediah Northern Highlands, CLan Chief's Mausoleum == The Mausoleum
# Pre-Kamasylvia update, Pre-Mediah update, Pre-Drieghan
# Added Abandoned Monastery, North Abandoned Quarry, Gehaku Plain, Serendia Western Gateway
# Removed Closed Western Gateway (became Serendia Western Gateway)
# Ignore fishing nodes (i.e. islands)
# 255CP soft cap

In [8]:
# Import conn_matrix.csv (basically final_node_names (clean) as a CSV file without row/column names)
# Connection matrix between all nodes (excluding islands for now)
conn_mat = np.loadtxt(open("conn_matrix.csv"), delimiter=",", skiprows=0)

# Connection matrices are symmetric in our case (two-way paths)
# https://stackoverflow.com/questions/42908334/checking-if-a-matrix-is-symmetric-in-numpy
def check_symmetric(a, tol=1e-8):
    return np.allclose(a, a.T, atol=tol)

check_symmetric(conn_mat)

True

In [9]:
# TODO
# Data mine bdodae by Dae for all nodes

# Function variables:
# Worker type, yield/price of yield, worker type, (worker housing?), worker travel distance, contribution points
#
# Constraints:
# Contribution points (CP)

In [10]:
# Getting CP (contribution points) values for each node
cp = nodes[['Name', 'CP']]

# Matching node names between two files
a = set(nodes['Name'])
nodes_names = pd.read_excel(io = "final_node_names.xlsx", sheet_name = "Sheet1")
b = set(nodes_names['Name'])
matches = a & b
cp = cp[cp['Name'].isin(matches)]

# Match main node name to its resource/subnode
balzor_nodes = balzor.iloc[0:150,]
a = set(balzor_nodes['Node'])
matches = list(a & b)

# Create subnodes to add to node matrix making sure to have unique duplicates (e.g. ***** and *****.1)
subnodes = balzor_nodes['Node'] + " - " + balzor_nodes['Main Material']

# https://stackoverflow.com/questions/2837409/how-to-append-count-numbers-to-duplicates-in-a-list-in-python
def rename_duplicates( old ):
    seen = {}
    for x in old:
        if x in seen:
            seen[x] += 1
            yield "%s.%d" % (x, seen[x])
        else:
            seen[x] = 0
            yield x

subnodes = list(rename_duplicates(subnodes))

# Create a zero submatrix for the added subnodes
z = pd.DataFrame(np.zeros((len(subnodes), len(final_node['Name']) + 1)))
z.columns = list(final_node.columns)
z['Name'] = subnodes


# https://stackoverflow.com/questions/39948757/how-to-delete-rows-in-python-pandas-dataframe-using-regular-expressions
patternDel = '(Island|island)'
filter = cp['Name'].str.contains(patternDel)
cp = cp[~filter]
cp = cp.reset_index(drop = True)



# Contribution points for subnodes
cp2 = pd.DataFrame({'Name' : subnodes,
                    'CP' : balzor_nodes['Subnode Cp']})
# Add to main node CP matrix
cp = cp.append(cp2, ignore_index = True)

# Use the connection matrix to make a cost-matrix with CP
cp_mat = np.diag(cp['CP'])

C:\Users\Christopher Wong\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [11]:
# Create a dataframe containing main nodes and subnodes (rows) and main nodes (columns)
y = final_node.append(z, ignore_index = True)

# Loop that marks connections between subnodes and their main node
# If node string matches, then put a one in [i,j] and [j,i]
# Subnodes start at index 172
for row in list(y.iloc[172:, :]['Name']):
    # Split string by hyphen and take everything before the space and hyphen
    row2 = re.split('\s-', row)[0]
    #print(row)
    for col in y.columns:
        # Removes the '.1' after duplicate names
        col2 = re.split('\.', col)[0]
        #print(col)
        if row2 == col2:
            y.loc[y['Name'] == row, col] = 1



In [12]:
# Creating a transposed copy of our subnode to main node connection matrix
profit = balzor_nodes['Silver Per Day']

# Setting a higher weight going to subnodes from their main node
# Arbitrary weight of 0.25
z2 = y.iloc[172:, :].set_index('Name').T @ (np.diag(profit * 0.25))
z2.index.name = 'Name'
z2.columns = subnodes


# Creating a zero matrix between subnodes (because subnodes are not connected to each other)
df2 = pd.DataFrame(np.zeros((len(subnodes), len(subnodes))), index = subnodes, columns = subnodes)

# Appending both matrices
z2 = z2.append(df2)
z2 = z2.rename_axis('Name').reset_index()

# Merge everything into a complete matrix including both connections of main nodes and subnodes
final = pd.merge(y, z2, on = 'Name')
final = final.set_index('Name')

In [13]:
# https://www.gurobi.com/documentation/8.0/quickstart_windows/py_example_mip1_py.html

# Multiplying by a diagonal matrix multiplies each row of our connection matrix by the CP needed to connect
cp_conn_mat = np.array(cp_mat @ final)

# We take the transpose to keep this notation:
# Row: Starting point
# Column: End point
distances = np.array(final)

# Setting distance = 1/profit for subnodes with resource yield
distances[:, 172:] = np.multiply(distances[:, 172:], 1/profit.values)
# Should it be symmetrical? Probably not, the subnodes give value to the main node.
#distances[172:, :] = distances[:, 172:].T

# Change this when we change distances to profits in ant_colony_modified.py
distances[distances == 0] = np.inf
distances = np.array(distances)


In [64]:
# final.columns.get_loc('Quint Hill')
final.iloc[26:27, 218:222]
distances[57:58, :]

def node_name(a):
    for x in a: print(final.columns[x])
node_name([73,100,47,229,62,228])


Abandonned Land
Isolated Sentry Post
Quint Hill
Quint Hill - Lead Ore
Quint Hill.1
Quint Hill - Birch


,Castle Ruins - Maple Timber,Khuruto Cave - Tin Ore,Old Dandelion - Birch Timber,Karanda Ridge - Silk Honey Grass
Name,,,,
Balenos Forest.1,0.0,0.0,0.0,0.0


In [59]:
import importlib
import ant_colony_modified #import the module here, so that it can be reloaded.
importlib.reload(ant_colony_modified)
from ant_colony_modified import AntColony # or whatever name you want.

# adjust distances starting from city (currently all set to 1)
# Weight according to subnode values
# 
# replace dead ends with back to town
# modify algorithm to keep searching after back to town

# Choose Velia as a starting location : start_node = 39
# Choose Calpheon as a starting location : start_node = 80
# Choose Heidel as a starting location : start_node = 154
# Choose Olivia as a starting location : start_node = 166
ant_colony = AntColony(distances, 80, 100, 10, 5, 0.9, alpha=1, beta=3)
shortest_path = ant_colony.run()

print ("shortest_path: {}".format(shortest_path))


{169, 170, 44, 46, 50, 73, 74, 75, 76, 77, 79, 80, 86, 87, 89, 92, 220, 95, 97, 100, 105, 106, 110, 111, 112, 115, 116, 246, 118, 121, 122}
shortest_path: ([(80, 92), (92, 44), (44, 46), (46, 220), (220, 80), 'End of path', (80, 87), (87, 75), (75, 53), (53, 45), (45, 222), (222, 68), (68, 223), (223, 80), 'End of path', (80, 73), (73, 86), (86, 100), (100, 47), (47, 229), (229, 62), (62, 228), (228, 80), 'End of path', (80, 95), (95, 105), (105, 77), (77, 121), (121, 122), (122, 115), (115, 116), (116, 89), (89, 170), (170, 98), (98, 171), (171, 159), (159, 161), (161, 157), (157, 149), (149, 139), (139, 199), (199, 144), (144, 200), (200, 80), 'End of path', (80, 110), (110, 76), (76, 112), (112, 79), (79, 55), (55, 99), (99, 123), (123, 64), (64, 241), (241, 59), (59, 242), (242, 80), 'End of path', (80, 97)], 0.05603448275862069)


In [18]:
# https://github.com/Akavall/AntColonyOptimization
# Added parenthesis around print statement
# Changed xrange to range (xrange was used in Python 2, changed to range in Python 3)

# Added start_node to arguments

import numpy as np

import importlib
import ant_colony_modified #import the module here, so that it can be reloaded.
importlib.reload(ant_colony_modified)
from ant_colony_modified import AntColony # or whatever name you want.


distances = np.array([[np.inf, np.inf, np.inf, 5, np.inf],
                      [np.inf, np.inf, 4, 8, np.inf],
                      [np.inf, 4, np.inf, 1, np.inf],
                      [5, 8, 1, np.inf, 2],
                      [np.inf, np.inf, np.inf, 2, np.inf]])


#distances = np.array([[np.inf, 2, 2, 5, 7],
#                      [2, np.inf, 4, 8, 2],
#                      [2, 4, np.inf, 1, 3],
#                      [5, 8, 1, np.inf, 2],
#                      [7, 2, 3, 2, np.inf]])

ant_colony = AntColony(distances, 4, 10, 10, 50, 0.95, alpha=1, beta=1)
shortest_path = ant_colony.run()
print ("shortest_path: {}".format(shortest_path))
#

([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3, 2), (2, 1), (1, 4), 'Dead end'], 7.0)
([(4, 3), (3